In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

import utils

# Ignore all warnings
warnings.filterwarnings("ignore")

# Download NLTK resources
utils.checkForNLTKResources()

trainDataset = "_data/train_essays.csv"
testDataset = "_data/test_essays.csv"
promtDataset = "_data/train_prompts.csv"

trainDf = pd.read_csv(trainDataset)
testDf = pd.read_csv(testDataset)
promptDf = pd.read_csv(promtDataset)

In [ ]:
print("--------trainDf-------")
print(trainDf.info())
print("\n")

print("--------testDf-------")
print(testDf.info())
print("\n")

print("--------promptDf-------")
print(promptDf.info())

In [ ]:
# prompt id distribution
sns.set(style="darkgrid")
sns.countplot(x='prompt_id', data=trainDf)
plt.title('Prompt ID Distribution')
plt.show()

In [ ]:
# target distribution
generated = trainDf['generated'].value_counts().to_dict()
print(f"Human: {generated[0]}")
print(f"AI: {generated[1]}")
sns.countplot(x='generated', data=trainDf)
plt.title('Target Distribution')
plt.show()

In [ ]:
# external datasets to complete the training set
externalLLMGeneratedTextsDf = pd.read_csv("_data/LLM-Mistral-7B-Instruct-texts/Mistral7B_CME_v7.csv")
print("------LLM Mistral 7B Dataset------")
print(externalLLMGeneratedTextsDf.info())
print("\n")

print("------LLM Mistral 7B Disitrbution------")
print(externalLLMGeneratedTextsDf['prompt_name'].value_counts())
print("\n")

promptNames = promptDf['prompt_name'].to_list()
externalLLMDf = externalLLMGeneratedTextsDf[externalLLMGeneratedTextsDf['prompt_name'].isin(promptNames)]
externalLLMDf.loc[:,'generated'] = 1
promptId_map_dict = {2: 0 , 12: 1} # matching prompt_id columns with the train dataset
externalLLMDf["prompt_id"] = externalLLMDf["prompt_id"].map(promptId_map_dict)
externalLLMDf = externalLLMDf.drop(columns=['prompt_name'])
print("------External Dataset------")
print(externalLLMDf.info())
print("\n")
print("------External Dataset Distribution------")
print(externalLLMDf['prompt_id'].value_counts())

# prepare train dataset to be concatenated with the external dataset
trainDf = trainDf.drop(columns=['id'])

In [ ]:
# concatenate the traind dataset with the external LLM dataset
newTrainDf = pd.concat([trainDf, externalLLMDf], ignore_index=True)
print("------Concatenated Dataset------")
print(newTrainDf['generated'].info())
print("\n")
print("------Concatenated Dataset Generated Distribution------")
print(newTrainDf['generated'].value_counts())
sns.countplot(x='generated', data=newTrainDf)
plt.title('Target Distribution')
plt.show()

In [ ]:
# verify if there are duplicates
print(f"Before dropping duplicates: {len(newTrainDf)}")
newTrainDf = newTrainDf.drop_duplicates(subset=['text'])
print(f"After dropping duplicates: {len(newTrainDf)}")

In [ ]:
textSample = newTrainDf['text'].iloc[2]
cleanedTextSample = utils.cleanText(textSample, True)
print(cleanedTextSample)

In [ ]:
textSample = newTrainDf['text'].iloc[2]
cleanedTextSample = utils.cleanText(textSample, False)
print(cleanedTextSample)